Importation des librairies utiles

In [7]:
import requests
import pandas as pd

Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [8]:
categories_list = [
    {"catcolor": "paleturquoise", "catid": 8, "catname": "Absence d'aménagement", "catname_en_US": "Missing of bicycle or pedestrian way"},
    {"catcolor": "seagreen", "catid": 3, "catname": "Aménagement mal conçu", "catname_en_US": "Bad designed way"},
    {"catcolor": "darkgreen", "catid": 4, "catname": "Défaut d'entretien", "catname_en_US": "Maintenance issue"},
    {"catcolor": "turquoise", "catid": 5, "catname": "Absence d'arceaux de stationnement", "catname_en_US": "Missing bicycle parking"},
    {"catcolor": "crimson", "catid": 2, "catname": "Véhicule ou objet gênant", "catname_en_US": "Annoying vehicle or object"},
    {"catcolor": "black", "catid": 9, "catname": "Accident, chute, incident", "catname_en_US": "Accident, incident or drop "},
    {"catcolor": "yellowgreen", "catid": 6, "catname": "Signalisation, marquage", "catname_en_US": "Traffic sign or labeling"},
    {"catcolor": "lightcoral", "catid": 7, "catname": "Incivilité récurrente sur la route", "catname_en_US": "Antisocial behaviour on the road"},
    {"catcolor": "blue", "catid": 10, "catname": "Vol ou dégradation de vélo", "catname_en_US": "Bike theft or vandalism"},
    {"catcolor": "grey", "catid": 100, "catname": "Autre", "catname_en_US": "Other"},
    {"catcolor": "orange", "catdisable": True, "catid": 50, "catname": "COVID - Distanciation physique impossible"},
    {"catcolor": "red", "catdisable": True, "catid": 51, "catname": "COVID - Aménagement fermé"},
    {"catcolor": "green", "catdisable": True, "catid": 52, "catname": "COVID - Voirie en période de confinement"},
    {"catcolor": "black", "catid": 11, "catname": "Éclairage public insuffisant", "catname_en_US": "Insufficient public lighting"}
]

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {cat["catname"]: str(cat["catid"]) for cat in categories_list}

# Création du dataframe des catégories avec les colonnes "id" et "catégorie"
df_categories = pd.DataFrame(categories_list).loc[:, ["catid", "catname"]]
# Renommer les colonnes
df_categories = df_categories.rename(columns={"catid": "id", "catname": "catégorie"})
# Changer l'index et trier par id
df_categories = df_categories.set_index("id").sort_index()
df_categories

,catégorie
id,
2,Véhicule ou objet gênant
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
8,Absence d'aménagement
9,"Accident, chute, incident"
10,Vol ou dégradation de vélo


Initialisation des paramètres

In [9]:
start_date = None #Default value
end_date = None #Default value
url = 'https://vigilo.bapav.org/get_issues.php'
category_filter = None  # Default value
location_filter = None  # Default value
status_filter = None  # Default value
approval_filter = None  # Default value
scope_filter = None # Default value
keyword_search = None  # Default value
data_limit = None  # Default value
# Filtre des catégories à afficher, exemple : category_filter = categories_dict["Absence d'aménagement"]
# donne la catégorie "Absence d'aménagement" 
category_filter = None

Fonction créant le dataframe

In [10]:
def data_collection(start_date, end_date, url, category_filter=None, location_filter=None,
                    status_filter=None, approval_filter=None, scope_filter=None,
                    keyword_search=None, data_limit=None):
    try:
        params = {
            'start_date': start_date,
            'end_date': end_date,
            'location_filter': location_filter,
            'status_filter': status_filter,
            'approval_filter': approval_filter,
            'scope_filter': scope_filter,
            'keyword_search': keyword_search,
            'data_limit': data_limit
        }
        resp = requests.get(url, params=params)
        resp.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        df = pd.DataFrame(resp.json())
        return df[df['categorie'] == category_filter] if category_filter else df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None

Création du dataframe

In [11]:
df = data_collection(start_date, end_date, url, category_filter, location_filter,
                     status_filter, approval_filter, scope_filter, keyword_search, data_limit)

test

In [12]:
df['categorie'].value_counts()

categorie
2      448
3      110
4       66
5       61
8       57
6       32
7       13
9        7
100      4
52       3
Name: count, dtype: int64